## Testing on Data
Opening the database files


~corpus folder

In [41]:
import os
import sys
import json

sys.path.append(os.getcwd())
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from dataloc import directory_path
from pathlib import Path

data_path = Path(directory_path)

print(data_path)

# full_path = os.path.join(directory_path, r'corpus\document2anumber.json')
# full_path = directory_path + r"\corpus\document2anumber.json"
full_path = data_path / "corpus" / "document2anumber.json"

print(full_path)

with open(full_path, 'r') as file:
    data = json.load(file)

..\..\wamex\data
..\..\wamex\data\corpus\document2anumber.json


# Open Domain Dictionary Files

In [5]:
Domain_Dictionary_Path = {"geological_timescales": "Domain_Dictionary/geological_timescales.txt",
                          "locations": "Domain_Dictionary/locations.txt",
                          "minerals": "Domain_Dictionary/minerals.txt",
                          "ores_deposits": "Domain_Dictionary/ores_deposits.txt",
                          "rocks": "Domain_Dictionary/rocks.txt",
                          "stratigraphy": "Domain_Dictionary/stratigraphy.txt"}

Domain_Dictionary = {}

for domain, path in Domain_Dictionary_Path.items():
    with open(path, "r") as file:
        info = file.read().splitlines()
        if domain == "geological_timescales":
            info = [i.lstrip().split()[0] for i in info]
        info = [i for i in info if i.strip()] # Make sure there are no empty strings
        Domain_Dictionary[domain] = info

        print(info)
        print(len(info))

# print(Domain_Dictionary["minerals"])

['Aalenian', 'Abereiddian', 'Acadian', 'Actonian', 'Adelaidean', 'Aegean', 'Aeronian', 'Aksayan', 'Aktastinian', 'Alaunian', 'Albertan', 'Albian', 'Aldingian', 'Alexandrian', 'Alportian', 'Altonian', 'Amgan', 'Animikean', 'Anisian', 'Aphebian', 'Aptian', 'Aquitanian', 'Aratauran', 'Archean', 'Archeozoic', 'Arenig', 'Arenigian', 'Arikareean', 'Aritan', 'Arnsbergian', 'Arowhanan', 'Artinskian', 'Arundian', 'Asbian', 'Ashgill', 'Asselian', 'Astian', 'Atdabanian', 'Atokan/Derryan', 'Atokan', 'Aurelucian', 'Austinian', 'Auversian', 'Awamoan', 'Ayusokkanian', 'Azoic', 'Baigendzinian', 'Bairnsdalian', 'Baishaean', 'Bajocian', 'Bala', 'Balan', 'Balcombian', 'Bananian', 'Baotan', 'Barremian', 'Barstovian', 'Bartonian', 'Bashkirian', 'Basin', 'Batesfordian', 'Bathonian', 'Batyrbayan', 'Begudian', 'Bendigonian', 'Berriasian', 'Bithynian', 'Black', 'Blackriveran', 'Blackriverian', 'Blancan', 'Bolindian', 'Bolsovian', 'Boomerangian', 'Bortonian', 'Botomian', 'Braxtonian', 'Bridgerian', 'Brigantian'

### geo ontology
extract info

In [6]:
# https://github.com/CGI-IUGS/timescale-data/tree/master/rdf ontology grabbed here

from rdflib import Graph

# Create a graph
g = Graph()

# Parse the data from the TTL file
g.parse("Domain_Dictionary/isc2020.ttl", format="ttl")

geo_subject_label_vocab = {}

query = """
PREFIX gts: <http://resource.geosciml.org/ontology/timescale/gts#>
PREFIX isc: <http://resource.geosciml.org/classifier/ics/ischart/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?subject ?label
WHERE {
  ?subject a ?type ;
            rdfs:label ?label ;
  FILTER (?type IN (
    gts:Super-Eon,
    gts:Eon, 
    gts:Era, 
    gts:Period, 
    gts:Sub-Period,
    gts:Epoch, 
    gts:Age
  ))
}
"""

# Execute the query
results = g.query(query)

# Process the results
for row in results:
    subject = row.subject
    label = row.label

    geo_subject_label_vocab[subject] = label
    

# Define the query to fetch all subjects and their labels based on [Eon, Era, Period, Epoch, Age]
geo_dict_scale = {}
time_periods = ["Eon", "Era", "Sub-Period", "Period", "Epoch", "Age"]

for time_period_type in time_periods:
    query = f"""
    PREFIX gts: <http://resource.geosciml.org/ontology/timescale/gts#>
    PREFIX isc: <http://resource.geosciml.org/classifier/ics/ischart/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?subject ?label ?broader ?narrower ?comment
    WHERE {{
      ?subject a gts:{time_period_type} ;
                rdfs:label ?label ;
                rdfs:comment ?comment .
        OPTIONAL {{ ?subject skos:broader ?broader }} .
        OPTIONAL {{ ?subject skos:narrower ?narrower }} .
    }}
    """
    # Execute the query
    results = g.query(query)

    # Dictionary to hold the combined comments
    combined_dict = {}
    # Process the results
    for row in results:
        subject = row.subject
        label = row.label
        broader = row.broader
        comment = row.comment
        narrower = row.narrower
  
        # Initialize the dictionary entry if it doesn't exist
        if (subject, label) not in combined_dict:
            combined_dict[(subject, label)] = {
                "broader": broader,
                "narrower": [],
                "comments": []
            }
        
        # Append the comment to the dictionary entry
        if comment not in combined_dict[(subject, label)]["comments"]:
            combined_dict[(subject, label)]["comments"].append(comment)

        # Append to narrower list if not already present
        if narrower and narrower not in combined_dict[(subject, label)]["narrower"]:
            combined_dict[(subject, label)]["narrower"].append(narrower)

    # Print the results
    for (subject, label), data in combined_dict.items():
        broader = data["broader"]
        comments = [str(c) for c in data["comments"]]
        
        # print(f"Subject: {subject}, Label: {label}, Broader: {broader}")
        # print(f"Comments: {comments}")
        # print(f"Narrower: {data['narrower']}")

    geo_dict_scale[time_period_type] = combined_dict


In [7]:
def count_numbers(list):
    count = 0
    for item in list:
        try:
            # Attempt to convert the item to a float
            float(item.replace('|', '').replace('+', ''))
            count += 1
        except ValueError:
            pass
    return count

def find_geo_time_bound(timeBounds):
    bound_count = count_numbers(timeBounds)
    if bound_count == 1:
        bound = timeBounds[-2]
        uncertainty = 0
    elif bound_count == 2:
        bound = timeBounds[-3]
        uncertainty = timeBounds[-2]
    else:
        bound = None
        uncertainty = None
    return bound, uncertainty

# Place the geological time scale dictionary into the Domain Dictionary json file format
time_periods = ["Age", "Epoch", "Period", "Sub-Period", "Era", "Eon"]

timescale_sorted = {"Eon": {}, "Era": {}, "Sub-Period": {}, "Period": {}, "Epoch": {}, "Age": {}}
prev_type = None
for time_period_type in time_periods:
    current_list = {}
    for (subject, label), data in geo_dict_scale[time_period_type].items():
        broader = data["broader"]
        comments = [str(c) for c in data["comments"]]

        label = str(label)
        # Get the broader and narrower labels
        broader_label = str(geo_subject_label_vocab[broader]) if broader else None
        narrower = [str(geo_subject_label_vocab[n]) for n in data["narrower"] if n]
        # Find the upper and lower bounds
        older_bound = comments[0].split(" ") if comments else None
        younger_bound = comments[-1].split(" ") if comments else None
        upperbound, upper_uncertainty = find_geo_time_bound(older_bound)
        lowerbound, lower_uncertainty = find_geo_time_bound(younger_bound)

        timescale_sorted[time_period_type][label] = {
            "broader": broader_label,
            "narrower": narrower,
            "upperbound" : upperbound,
            "upper_uncertainty" : upper_uncertainty,
            "lowerbound" : lowerbound,
            "lower_uncertainty" : lower_uncertainty
        }

print(timescale_sorted)

{'Eon': {'Archean Eon': {'broader': 'Precambrian Supereon', 'narrower': ['Eoarchean Era', 'Mesoarchean Era', 'Neoarchean Era', 'Paleoarchean Era'], 'upperbound': '-4000', 'upper_uncertainty': 0, 'lowerbound': '-2500', 'lower_uncertainty': 0}, 'Hadean Eon': {'broader': 'Precambrian Supereon', 'narrower': [], 'upperbound': '-4567', 'upper_uncertainty': '+|-1', 'lowerbound': '-4000', 'lower_uncertainty': 0}, 'Phanerozoic Eon': {'broader': None, 'narrower': ['Cenozoic Era', 'Mesozoic Era', 'Paleozoic Era'], 'upperbound': '-541.0', 'upper_uncertainty': '+|-1.0', 'lowerbound': '-0.0', 'lower_uncertainty': 0}, 'Proterozoic Eon': {'broader': 'Precambrian Supereon', 'narrower': ['Mesoproterozoic Era', 'Neoproterozoic Era', 'Paleoproterozoic Era'], 'upperbound': '-2500', 'upper_uncertainty': 0, 'lowerbound': '-541.0', 'lower_uncertainty': '+|-1.0'}}, 'Era': {'Cenozoic Era': {'broader': 'Phanerozoic Eon', 'narrower': ['Neogene Period', 'Paleogene Period', 'Quaternary Period'], 'upperbound': '-66.

In [8]:
# Not Finished, decided to do different method

# # Place the geological time scale dictionary into the Domain Dictionary json file format
# time_periods = ["Eon", "Era", "Sub-Period", "Period", "Epoch", "Age"]

# timescale_sorted = {}
# previous_list = {}
# for time_period_type in time_periods:
#     current_list = {}
#     for (subject, label), data in geo_dict_scale[time_period_type].items():
#         broader = data["broader"]
#         comments = [str(c) for c in data["comments"]]

#         # print(f"Subject: {subject}, Label: {label}, Broader: {broader}")
#         # print(f"Comments: {comments}")

#         label = str(label)
            
#         if time_period_type != "Eon":
#             broader_label = str(geo_subject_label_vocab[broader])
#             if broader_label not in current_list:
#                 current_list[broader_label] = {
#                     "type": time_period_type,
#                     label: {
#                         "upperbound" : None,
#                         "upper_uncertainty" : 0,
#                         "lowerbound" : None,
#                         "lower_uncertainty" : 0
#                     }
#                 }
#             else:
#                 current_list[broader_label][label] = {
#                     "upperbound" : None,
#                     "upper_uncertainty" : 0,
#                     "lowerbound" : None,
#                     "lower_uncertainty" : 0
#                 }

#             if label in previous_list:
#                 current_list[broader_label][label] = previous_list[label]
#                 print(previous_list[label])

#         else:
#             timescale_sorted

#     if (time_period_type == "Sub-Period"):
#         current_list.update(previous_list)

#     previous_list = current_list
#     print(current_list)

# print(current_list)



In [9]:
# import rdflib

# with open("Domain_Dictionary/geological_timescales.txt", "r") as file:
#     geological_timescale_text = file.read()

# print(geological_timescale_text)
# #  eons eras periods epochs ages 

# # Extract the geological timescales words


# # Extract the geological timescales
# geological_timescales = geological_timescale_text.split("\n")
# geo_temporal_scale = {}
# geo_temporal_scale["eons"] = []


In [10]:
from dataloc import directory_path

data_path = directory_path
print(Path(data_path) / "corpus" / "document2anumber.json")
print(os.path.join(directory_path, r'corpus\document2anumber.json'))

with open(Path(data_path) / "corpus" / "document2anumber.json", 'r') as file:
    # os.path.join(directory_path, r'corpus\document2anumber.json'), 'r') as file:
    data = json.load(file)

with open(Path(data_path) / "corpus" / "document2keywords.json", 'r') as file:
    # os.path.join(directory_path, r'corpus/document2keywords.json'), 'r') as file:
    datafd = json.load(file)

with open(Path(data_path) / "corpus" / "keyword2documents.json", 'r') as file:
    # os.path.join(directory_path, r'corpus/keyword2documents.json'), 'r') as file:
    datakeys = json.load(file)

for i, d in enumerate(data):
    if i > 0:
        break
    print("document2anumber.json:")
    print(d, data[d])

print()

for i, d in enumerate(datafd):
    if i > 0:
        break
    print("document2keywords.json:")
    print(d, datafd[d])

print()

for i, d in enumerate(datakeys):
    if i > 0:
        break
    print("keyword2documents.json:")
    print(d, datakeys[d])

print()

print("~~~~~~'a073533_nf_e09_1004_2006a_12145691.pdf' testing:")
print(data['a073533_nf_e09_1004_2006a_12145691.pdf'])
print(datafd['a073533_nf_e09_1004_2006a_12145691.pdf'])

print()


with open(Path(data_path) / "wamex_metadata" / "073533.json", 'r') as file:
    meta_data = json.load(file)

print("073533.json testing:")
print(meta_data)



..\..\wamex\data\corpus\document2anumber.json
..\..\wamex\data\corpus\document2anumber.json
document2anumber.json:
a087761_e28_1833_2010_16911280.pdf 087761

document2keywords.json:
a087761_e28_1833_2010_16911280.pdf {'laverton-karonie greenstone belt': {'textrank': 0, 'tf_idf': 0.1870187238}, 'felsic schists shales': {'textrank': 0.0322580645, 'tf_idf': 0.1745545671}, 'mt monger gold': {'textrank': 0.0418788908, 'tf_idf': 0.3740374477}, 'for gold': {'textrank': 0.0470761702, 'tf_idf': 0.2100083723}, 'dolerite': {'textrank': 0.0322580645, 'tf_idf': 0.0453549253}, 'nickel': {'textrank': 0.0322580645, 'tf_idf': 0.0429806772}, 'significant gold': {'textrank': 0.0248490117, 'tf_idf': 0.0708358369}, 'flexure within host greenstone sequences': {'textrank': 0.0182462575, 'tf_idf': 0.1798837496}, 'regolith': {'textrank': 0.0322580645, 'tf_idf': 0.0987627594}, 'ppb au': {'textrank': 0.0322580645, 'tf_idf': 0.0575461992}, 'cowarna rocks project': {'textrank': 0.0226372383, 'tf_idf': 0.4128977757

### Opening PDF 

In [11]:
# ! pip install pypdf
# https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
from pypdf import PdfReader 

reader = PdfReader(Path(directory_path) / "wamex_pdf" / "a073533_nf_e09_1004_2006a_12145691.pdf")
    # os.path.join(directory_path, r'wamex_pdf/a073533_nf_e09_1004_2006a_12145691.pdf'))

print(len(reader.pages))

for i in range(len(reader.pages)):
        print(f"Page {i+1}:")
        print(reader.pages[i].extract_text())


23
Page 1:
  
 
 
 
 
 
32 Kings Park Rd West Perth 6005 
 
 
Midwest Corporation Limited 
 
New Forest Project 
Annual Report for E70 / 1004 
 
26 September 2005 – 25 September 2006 
 
 
 
 
 
 
 
 
 
Author:  David Broomfield  
B.Sc. (Hons), Dip. Ed. 
Validated: David Broomfield 
B.Sc. (Hons), Dip. Ed. 
Date:  20th October 2006 
Page 2:
Midwest Corporation Limited 
Annual Report E09/1004 September 2006  
 
 
 
 
 
 
TABLE OF CONTENTS 
                                   PAGE 
EXECUTIVE SUMMARY.....................................................................................................................1 
1.0 BIBLIOGRAPHIC DATA SHEET..........................................................................................2 
2.0 INTRODUCTION......................................................................................................................3 
2.1 NEW FOREST LOCATION AND ACCESS.........................................................................................

In [12]:
filepath = Path(directory_path) / "wamex_xml"
content = {}
for root, dirs, files in os.walk(filepath):
    for file in files:
        try:
            with open(filepath / file, 'r') as f:
                content[file] = json.load(f)
        except:
            pass

In [13]:
# ! python -m spacy download en_core_web_sm
# ! pip install spacy

In [14]:
import re
import spacy

# Spacy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# # nlp.Defaults.stop_words = 
# print(nlp.Defaults.stop_words)
# for word in nlp.Defaults.stop_words:
#     print(word)


# Perform Stopwords Removal, Lemmatisation and Tokenisation
def spacy_processing(x):
    doc = nlp(x)
    x = [word.lemma_ for word in doc] # if not word.is_stop]
    return x

# Function to preprocess the data
def preprocess_data(sentence_list):
    output_list = []
    for sentence in sentence_list:
        sentence = sentence.lower()                     # Case folding
        sentence = re.sub(r'[^\w\s]', ' ', sentence)    # Remove punctuation
        sentence = re.sub(r"\s+", " ", sentence)        # Remove extra spaces
        tokens = spacy_processing(sentence)             # Remove stopwords / Lemmatisation / Tokenise
        output_list.append(tokens)
    return output_list

In [15]:
pages = []

for i in range(len(reader.pages)):
    pages.append(reader.pages[i].extract_text())

output = preprocess_data(pages)
for i in range(len(output)):
    print(f"Page {i+1}:")
    print(output[i])

Page 1:
[' ', '32', 'king', 'park', 'rd', 'west', 'perth', '6005', 'midwest', 'corporation', 'limit', 'new', 'forest', 'project', 'annual', 'report', 'for', 'e70', '1004', '26', 'september', '2005', '25', 'september', '2006', 'author', 'david', 'broomfield', 'b', 'sc', 'hon', 'dip', 'ed', 'validate', 'david', 'broomfield', 'b', 'sc', 'hon', 'dip', 'ed', 'date', '20th', 'october', '2006']
Page 2:
['midwest', 'corporation', 'limit', 'annual', 'report', 'e09', '1004', 'september', '2006', 'table', 'of', 'content', 'page', 'executive', 'summary', '1', '1', '0', 'bibliographic', 'datum', 'sheet', '2', '2', '0', 'introduction', '3', '2', '1', 'new', 'forest', 'location', 'and', 'access', '3', '2', '2', 'new', 'forest', 'tenure', '5', '3', '0', 'new', 'forest', 'project', 'history', '8', '4', '0', 'geology', '9', '4', '1', 'regional', 'geology', 'of', 'the', 'melia', 'creek', 'tenement', '9', '4', '2', 'structure', 'and', 'metamorphism', '9', '4', '3', 'iron', 'mineralisation', '10', '4', '4'

In [16]:
print(pages[2].split()[177])

banded


In [17]:
import re
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Perform Stopwords Removal, Lemmatisation and Tokenisation
def spacy_processing(x):
    doc = nlp(x)
    x = [word.lemma_ for word in doc] # if not word.is_stop]
    return x

# Function to preprocess the data
def preprocess_data(sentence_list):
    output_list = []
    for sentence in sentence_list:
        sentence = sentence.lower()                     # Case folding
        sentence = re.sub(r'[^\w\s]', ' ', sentence)    # Remove punctuation
        sentence = re.sub(r"\s+", " ", sentence)        # Remove extra spaces
        tokens = spacy_processing(sentence)             # Lemmatisation / Tokenise
        output_list.append(tokens)
    return output_list

def preprocess_word(word):
    word = word.lower()                     # Case folding
    word = re.sub(r'[^\w\s]', ' ', word)    # Remove punctuation
    word = re.sub(r"\s+", " ", word)        # Remove extra spaces
    return word

In [18]:
# ! python -m spacy download en_core_web_sm

In [19]:
import spacy
import re

# Load the spaCy model
sp_sm = spacy.load('en_core_web_sm')

def spacy_large_ner(document):
    # Extract named entities using spaCy
    entities = {(ent.text.strip(), ent.label_) for ent in sp_sm(document).ents}
    
    # Extract geological timescales
    pattern = r'\b\d+(?:[\.,]\d+)?\s*(?:Ma|ka|Ga|MYA|KYA)\b'
    matches = re.finditer(pattern, document, re.IGNORECASE)
    geological_timescales = {(match.group().strip(), 'GEOLOGICAL_TIME') for match in matches}
    
    # Combine geological timescales with named entities
    all_entities = entities.union(geological_timescales)
    
    return all_entities

# Sample text from geological surveys
text = """
This report has been prepared as an investigation of the Mt Aubrey tenement, as part of Midwest’s New
Forest project in the Murchison Region of Western Australia. The report is presented as an Annual Report
to be submitted to the Department of Industry and Resources as part of the conditions of the granting of
E09/1004 and covers the period from the 26 September 2005 to the 25 September 2006. We also have dates like
50 Ma and 2000 ka which are geological timescales.
"""

# Extract entities and geological timescales
print(spacy_large_ner(text))

# for a in content:
#     print(a)
#     for b in content[a]:
#         print(b)
#         print(spacy_large_ner(b))


{('New\nForest', 'ORG'), ('50 Ma', 'GEOLOGICAL_TIME'), ('the Department of Industry and Resources', 'ORG'), ('2000 ka', 'GEOLOGICAL_TIME'), ('the 25 September 2006', 'DATE'), ('50', 'CARDINAL'), ('the 26 September 2005', 'DATE'), ('the Murchison Region of Western Australia', 'LOC'), ('Midwest', 'LOC'), ('2000', 'DATE')}


In [25]:
import spacy
import re
from dateutil import parser

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def format_ner_output(text):
    # Tokenize the text using spaCy
    doc = nlp(text.strip())
    
    # Define the geological timescales pattern
    timescale_pattern = r'\b\d+(?:[\.,]\d+)?\s*(?:Ma|ka|Ga|MYA|KYA)\b'
    
    # Extract geological timescales and their positions
    geological_timescales = re.finditer(timescale_pattern, text, re.IGNORECASE)
    
    # Create a dictionary to store geological timescale spans
    geo_timescale_spans = set()

    for match in geological_timescales:
        start, end = match.span()
        geo_timescale_spans.add((start, end))
    
    # Prepare to format the output
    tokens_with_labels = []

    # Process tokens and assign labels
    for token in doc:
        token_text = token.text.strip()
        token_start = token.idx + 1
        token_end = token_start + len(token_text)
        
        # Check if token is part of a geological timescale
        token_label = 'O'

        # Check if token is a recognized entity
        if token.ent_type_:
            token_label = token.ent_type_
        

        for start, end in geo_timescale_spans:
            if token_start >= start and token_end <= end:
                if (token_start == start):
                    token_label = f'B-GEOLOGICAL_TIME'
                else:
                    token_label = f'I-GEOLOGICAL_TIME'
                break
        
        tokens_with_labels.append(f"{token_text} {token_label}")
    
    # Format the output
    output = '\n'.join(tokens_with_labels)
    
    return output


def extract_relationship_entities(doc):
    relationships = []
    
    for token in doc:
        if token.dep_ in ('attr', 'dobj', 'nsubj'):
            head = token.head
            if head.dep_ in ('ROOT', 'attr', 'dobj'):
                relationship = {
                    'entity': token.text,
                    'relationship': head.text
                }
                relationships.append(relationship)
    
    return relationships

# Sample text from geological surveys
text = """
This report has been prepared as an investigation of the Mt Aubrey tenement, as part of Midwest’s New
Forest project in the Murchison Region of Western Australia. The report is presented as an Annual Report
to be submitted to the Department of Industry and Resources as part of the conditions of the granting of
E09/1004 and covers the period from the 26 September 2005 to the 25 September 2006.
"""

# Format the NER output
ner = format_ner_output(text)
print(ner)

# Extract temporal information
doc = nlp(text)

# Extract relationship entities
relationships = extract_relationship_entities(doc)
print("\nRelationship Entities:")
for rel in relationships:
    print(f"Entity: {rel['entity']}, Relationship: {rel['relationship']}")

This O
report O
has O
been O
prepared O
as O
an O
investigation O
of O
the O
Mt O
Aubrey O
tenement O
, O
as O
part O
of O
Midwest LOC
’s O
New ORG
 ORG
Forest ORG
project O
in O
the LOC
Murchison LOC
Region LOC
of LOC
Western LOC
Australia LOC
. O
The O
report O
is O
presented O
as O
an O
Annual O
Report O
 O
to O
be O
submitted O
to O
the ORG
Department ORG
of ORG
Industry ORG
and ORG
Resources ORG
as O
part O
of O
the O
conditions O
of O
the O
granting O
of O
 O
E09/1004 O
and O
covers O
the O
period O
from O
the DATE
26 DATE
September DATE
2005 DATE
to O
the DATE
25 DATE
September DATE
2006 DATE
. O

Relationship Entities:


### LOAD NER AND RUN

In [48]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

save_directory = "./NER/saved_model"
# Load the tokenizer and model from the saved directory
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory).to(device)

def tokenize_data(texts, tokenizer, max_length=256):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

def predict_entities(texts, model, tokenizer):
    inputs = tokenize_data(texts, tokenizer)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation
        outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# # Predict geological entities
# predictions = predict_entities(geo_texts, model, tokenizer)

# # Print the results
# print(predictions)
# for text, prediction in zip(geo_texts, predictions):
#     print(f"Text: {text}")
#     print(f"Prediction: {prediction}")
#     print()

In [49]:
def NER_tagging(content, model, tokenizer):
    ner_tagged = {}
    for a in content:
        ner_tagged[a] = {}
        for b in content[a]:
            ner_tagged[a][b] = predict_entities(b, model, tokenizer)

    return ner_tagged

# Perform NER tagging
# ner_tagged = NER_tagging(content, model, tokenizer)

Size of variables
- Content Shape - {filename: [sentence,...]}
- NER_tagging shape - {filename: {sentence: [tagged_sentence,...]}}

In [58]:
# Write to JSON file
# with open("Results/NER_tagged.json", "w") as json_file:
#     json.dump(ner_tagged, json_file)

# Load the NER tagged data
with open(Path() / "Results" / "ner_tagged.json", 'r') as file:
    ner_tagged = json.load(file)

### RE

In [21]:
# https://huggingface.co/models?other=relation-extraction

### REBEL Model

In [22]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# https://www.superteams.ai/blog/knowledge-graph-from-unstructured-text
# https://github.com/gswycf/Joint-Extraction-of-Entities-and-Relations-Based-on-a-Novel-Tagging-Scheme
# https://academic.oup.com/jamia/article/20/5/828/727128?login=false
# https://dl.acm.org/doi/fullHtml/10.1145/3462475
# https://huggingface.co/Babelscape/mrebel-large

def extract_triplets_typed(text):
    triplets = []
    relation = ''
    text = text.strip()
    current = 'x'
    subject, relation, object_, object_type, subject_type = '','','','',''

    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").split():
        if token == "<triplet>" or token == "<relation>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="en_XX", tgt_lang="tp_XX")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large").to(device)
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 2,
    "num_return_sequences": 2,
    "forced_bos_token_id": None,
}

# # Text to extract triplets from
# text = """
# This report has been prepared as an investigation of the Mt Aubrey tenement, as part of Midwest’s New Forest project in the Murchison Region of Western Australia. The report is presented as an Annual Report to be submitted to the Department of Industry and Resources as part of the conditions of the granting of E09/1004 and covers the period from the 26 September 2005 to the 25 September 2006.
# """
# # Tokenizer text
# model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

# # Generate
# generated_tokens = model.generate(
#     model_inputs["input_ids"].to(model.device),
#     attention_mask=model_inputs["attention_mask"].to(model.device),
#     decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
#     **gen_kwargs,
# )

# # Extract text
# decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

# # Extract triplets
# for idx, sentence in enumerate(decoded_preds):
#     print(f'Prediction triplets sentence {idx}')
#     print(extract_triplets_typed(sentence))


In [23]:
relation_extraction = {}
for c in content:
    relation_extraction[c] = []
    for content_text in content[c]:
        model_inputs = tokenizer(content_text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
        generated_tokens = model.generate(
            model_inputs["input_ids"].to(model.device),
            attention_mask=model_inputs["attention_mask"].to(model.device),
            decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
            **gen_kwargs,
        )
        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
        # for idx, sentence in enumerate(decoded_preds):
        #     print(f'Prediction triplets sentence {idx}')
        #     print(extract_triplets_typed(sentence))
        relation_extraction[c].append(decoded_preds)
    break

In [24]:
for c in relation_extraction:
    print(c)
    for r in relation_extraction[c]:
        for content_text in r:
            print(extract_triplets_typed(content_text))

a075237_ep_a_all_2007_10582280.json
[{'head': 'Canning Highway Mt.', 'head_type': 'loc', 'type': 'length', 'tail': '2 907', 'tail_type': 'num'}]
[{'head': 'Canning Highway Mt.', 'head_type': 'loc', 'type': 'maintained by', 'tail': 'GME Resources Ltd', 'tail_type': 'org'}]
[{'head': 'NiWest', 'head_type': 'org', 'type': 'headquarters location', 'tail': 'Perth', 'tail_type': 'loc'}]
[{'head': 'Pleasant WA 6153', 'head_type': 'misc', 'type': 'inception', 'tail': 'June 2007', 'tail_type': 'date'}]
[{'head': 'Eucalyptus Bore Project', 'head_type': 'loc', 'type': 'located in the administrative territorial entity', 'tail': 'Kookynie', 'tail_type': 'loc'}]
[{'head': 'Eucalyptus Bore Project', 'head_type': 'loc', 'type': 'location', 'tail': 'Murrin Murrin mine', 'tail_type': 'loc'}]
[{'head': 'Kookynie - Mt.', 'head_type': 'loc', 'type': 'named after', 'tail': 'Mt.', 'tail_type': 'loc'}]
[{'head': 'Kookynie - Mt.', 'head_type': 'loc', 'type': 'located on terrain feature', 'tail': 'Mt.', 'tail_t